In [17]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [19]:
print(os.listdir("/gdrive/My Drive/Colab Notebooks/ANN/BANANA"))

['ripe', 'unripe', 'overripe', 'rotten']


In [20]:
DATADIR = '/gdrive/My Drive/Colab Notebooks/ANN/BANANA'
CATEGORIES =["overripe","ripe","rotten","unripe"]
IMG_SIZE = 100

In [21]:
all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                print("hata")
create_all_data()

In [22]:
print("toplam görüntü sayısı: ",len(all_data))

toplam görüntü sayısı:  13478


In [23]:
X = []
y = []
for features, label in all_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE * IMG_SIZE * 3) # Düzleştirme işlemi
y = np.array(y)

In [24]:
# Veriyi bölme
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)


In [25]:
# Etiketleri one-hot kodlama
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)
y_val = to_categorical(y_val, num_classes=4)

In [26]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Önceden eğitilmiş EfficientNetB0 modelini yükleme
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Önceden eğitilmiş modelin katmanlarını dondurma
base_model.trainable = False

# EfficientNetB0'ın üzerine sıralı bir model oluşturma
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(4, activation='softmax'))

# Optimizasyon Algoritmasını Ayarla
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Veri Artırma
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
x_train = x_train.reshape((-1, IMG_SIZE, IMG_SIZE, 3))
x_val = x_val.reshape((-1, IMG_SIZE, IMG_SIZE, 3))




# Veri artırma jeneratörünü uygun hale getir
datagen.fit(x_train)

# Modeli eğitme
model.fit(datagen.flow(x_train, y_train, batch_size=32), validation_data=(x_val, y_val), epochs=10)


Epoch 1/10
266/266 [==============================] - 308s 1s/step - loss: 0.3910 - accuracy: 0.8549 - val_loss: 0.2313 - val_accuracy: 0.9163
Epoch 2/10
266/266 [==============================] - 299s 1s/step - loss: 0.2423 - accuracy: 0.9124 - val_loss: 0.2076 - val_accuracy: 0.9237
Epoch 3/10
266/266 [==============================] - 295s 1s/step - loss: 0.2052 - accuracy: 0.9224 - val_loss: 0.1957 - val_accuracy: 0.9322
Epoch 4/10
266/266 [==============================] - 297s 1s/step - loss: 0.1819 - accuracy: 0.9335 - val_loss: 0.1999 - val_accuracy: 0.9343
Epoch 5/10
266/266 [==============================] - 298s 1s/step - loss: 0.1760 - accuracy: 0.9351 - val_loss: 0.1569 - val_accuracy: 0.9407
Epoch 6/10
266/266 [==============================] - 300s 1s/step - loss: 0.1631 - accuracy: 0.9408 - val_loss: 0.1539 - val_accuracy: 0.9481
Epoch 7/10
266/266 [==============================] - 295s 1s/step - loss: 0.1534 - accuracy: 0.9429 - val_loss: 0.1419 - val_accuracy: 0.9460

In [27]:
# Test seti üzerinde değerlendirme
x_test = x_test.reshape((-1, IMG_SIZE, IMG_SIZE, 3))
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test, axis=1)


127/127 [==============================] - 68s 522ms/step


In [28]:
# Performans metrikleri
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")


In [29]:
# Sonuçları gösterme
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))

Accuracy:  0.9495548961424333
Precision:  0.9499919458489734
Recall:  0.9495548961424333
F1 Score:  0.9495223352401916

Confusion Matrix:
 [[ 769   47    6    0]
 [  16 1140   30    7]
 [  26   13 1314    5]
 [   0    5   49  617]]

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.94      0.94       822
           1       0.95      0.96      0.95      1193
           2       0.94      0.97      0.95      1358
           3       0.98      0.92      0.95       671

    accuracy                           0.95      4044
   macro avg       0.95      0.94      0.95      4044
weighted avg       0.95      0.95      0.95      4044
